# Arch and Freq Invariance

This notebook shows the architecture and cpu frequency invariance features of EAS.

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os

import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
#from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
androidsdk_path=os.path.join(os.getcwd(), "../../../android-sdk")
my_target_conf = {
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : androidsdk_path,
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.65.34.1',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },
    
    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    #"host"        : '192.168.1.101',
    "host"        : '10.169.36.74',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
#     "rtapp-calib" : {
#         '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
#     }
}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
         ],
         "buffsize" : 10240
    },

    "results_dir" : "arch_and_freq_invariance",
}

In [5]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

03:09:11  INFO    : Using base path: /home/steven/lisa/lisa-github
03:09:11  INFO    : Loading custom (inline) target configuration
03:09:11  INFO    : Loading custom (inline) test configuration
03:09:11  INFO    : External tools using:
03:09:11  INFO    :    ANDROID_HOME: /home/steven/lisa/lisa-github/ipynb/workshop/../../../android-sdk
03:09:11  INFO    :    CATAPULT_HOME: /home/steven/lisa/lisa-github/tools/catapult
03:09:11  INFO    : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
03:09:11  INFO    : Connecting Android target [0123456789ABCDEF]
03:09:11  INFO    : Connection settings:
03:09:11  INFO    :    {'device': '0123456789ABCDEF'}
03:09:12  INFO    : Initializing target workdir:
03:09:12  INFO    :    /data/local/tmp/devlib-target
03:09:14  INFO    : Attempting to read energy model from target
03:09:14  INFO    : Topology:
03:09:14  INFO    :    [[0, 1, 2, 3], [4, 5, 6, 7]]
03:09:15  INFO    : Loading default EM:
03:09:15  INFO    :    /home/steven/lisa/lisa-github/lib

# Workload configuration

In [39]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        
        'task_p50': Periodic(
            period_ms=16,         # period
            duty_cycle_pct=20,     # duty cycle
            duration_s=10,         # duration    
            cpus=str(target.bl.bigs[0])   # pinned on first big CPU
        ).get(),
        
        'task_p50_l': Periodic(
            period_ms=16,         # period
            duty_cycle_pct=20,     # duty cycle
            duration_s=10,         # duration    
            cpus=str(target.bl.littles[0])   # pinned on first big CPU
        ).get(),
        
        'task_r20': Ramp(
            start_pct=5,           # intial load
            end_pct=95,            # end load
            delta_pct=10,          # load % increase...
            time_s=1,              # ... every 1[s]            
        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

04:09:38  INFO    : Setup new workload simple
04:09:38  INFO    : Workload duration defined by longest task
04:09:38  INFO    : Default policy: SCHED_OTHER
04:09:38  INFO    : ------------------------
04:09:38  INFO    : task [task_p50], sched: using default policy
04:09:38  INFO    :  | loops count: 1
04:09:38  INFO    : + phase_000001: duration 10.000000 [s] (625 loops)
04:09:38  INFO    : |  period    16000 [us], duty_cycle  20 %
04:09:38  INFO    : |  run_time   3200 [us], sleep_time  12800 [us]
04:09:38  INFO    : |  CPUs affinity: 4
04:09:38  INFO    : ------------------------
04:09:38  INFO    : task [task_p50_l], sched: using default policy
04:09:38  INFO    :  | loops count: 1
04:09:38  INFO    : + phase_000001: duration 10.000000 [s] (625 loops)
04:09:38  INFO    : |  period    16000 [us], duty_cycle  20 %
04:09:38  INFO    : |  run_time   3200 [us], sleep_time  12800 [us]
04:09:38  INFO    : |  CPUs affinity: 0
04:09:38  INFO    : ------------------------
04:09:38  INFO    :

# Workload execution round 1

In [40]:
# Set performance governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('performance')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target LITTLE CPU max CPUfreq:%s\n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target big CPU max CPUfreq:%s \n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

04:09:39  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
04:09:39  INFO    : Target current governor: performance
04:09:39  INFO    : Target LITTLE CPU max CPUfreq:1844000
		    Target LITTLE CPU current CPUfreq: 1844000
04:09:40  INFO    : Target big CPU max CPUfreq:2362000 
		    Target big CPU current CPUfreq: 2362000


In [41]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

04:09:40  INFO    : #### Setup FTrace
04:09:45  INFO    : #### Start energy sampling
04:09:46  INFO    : #### Start RTApp execution
04:09:46  INFO    : Workload execution START:
04:09:46  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
04:09:58  INFO    : #### Read energy consumption: /home/steven/lisa/lisa-github/results/20190308_150911/energy.json
04:09:58  ERROR   : /usr/bin/iio-capture terminated for Device1 (iio:device1)
04:09:58  ERROR   : [Unable to create IIO context
]
04:09:58  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
04:09:58  ERROR   : [Unable to create IIO context
]
04:09:58  INFO    : #### Stop FTrace
04:09:58  INFO    : #### Save FTrace: /home/steven/lisa/lisa-github/results/20190308_150911/trace.dat
04:10:03  INFO    : #### Save platform description: /home/steven/lisa/lisa-github/results/20190308_150911/platform.json


# Collected results

In [42]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

04:10:03  INFO    : Content of the output folder /home/steven/lisa/lisa-github/results/20190308_150911


total 48628
drwxrwxr-x 1 steven steven      310  3月  8 16:10 .
drwxrwxr-x 1 steven steven     2466  3月  8 15:09 ..
-rw-rw-r-- 1 steven steven        2  3月  8 16:09 energy.json
-rw-rw-r-- 1 steven steven        2  3月  8 16:09 energy_stats.json
-rw-rw-r-- 1 steven steven      444  3月  8 16:09 output.log
-rw-rw-r-- 1 steven steven     1526  3月  8 16:10 platform.json
-rw-r--r-- 1 steven steven    77660  3月  8 16:09 rt-app-task_p50-1.log
-rw-r--r-- 1 steven steven    77660  3月  8 16:09 rt-app-task_p50_l-2.log
-rw-r--r-- 1 steven steven    12560  3月  8 16:09 rt-app-task_r20-0.log
-rw-r--r-- 1 steven steven     3750  3月  8 16:09 simple_00.json
-rw-r--r-- 1 steven steven 49602560  3月  8 16:10 trace.dat
drwxrwxr-x 1 steven steven     2664  3月  8 15:32 .trace.dat.cache


In [43]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

04:10:03  INFO    : Generated RTApp JSON file:


In [44]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

04:10:03  INFO    : Energy: /home/steven/lisa/lisa-github/results/20190308_150911/energy.json


{}


In [45]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

04:10:03  INFO    : Platform description: /home/steven/lisa/lisa-github/results/20190308_150911/platform.json


# Trace inspection

In [46]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [47]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50','task_p50_l', "task_r20" ])

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7f4c7de8d290> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7f4c7de8d1d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.dequeue_task_fair object at 0x7f4c7de8d710> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7f4c7de8d7d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blo

In [48]:
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_task:util_avg',
      'sched_load_avg_task:load_avg'
    ],
    filters={
        'comm': ['task_p50']
    },
    drawstyle='steps-post',
    per_line=1,
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_cpu:util_avg',
      'sched_load_avg_cpu:load_avg'
    ],
    filters={
        'cpu': [ 4 ]
    },
    drawstyle='steps-post',
    per_line=1,
    xlim=(1, 16),
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()

""
""


""
""


In [50]:
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_task:util_avg',
      'sched_load_avg_task:load_avg'
    ],
    filters={
        'comm': ['task_p50_l']
    },
    drawstyle='steps-post',
    per_line=1,
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_cpu:util_avg',
      'sched_load_avg_cpu:load_avg'
    ],
    filters={
        'cpu': [ 0 ]
    },
    drawstyle='steps-post',
    per_line=1,
    xlim=(1, 16),
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()

""
""


""
""


# Workload Execution Round 2

In [52]:
# Set userspace governor with lowest CPUfreq
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('userspace')

LITTLE_max_freq=target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq")
big_min_freq=target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_min_freq")


target.write_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed", LITTLE_max_freq)
target.write_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_setspeed", 1805000)

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU min CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU min CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

04:19:40  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
04:19:41  INFO    : Target current governor: userspace
04:19:41  INFO    : Target big CPU min CPUfreq:903000
		    Target big CPU current CPUfreq: 1805000
04:19:41  INFO    : Target LITTLE CPU min CPUfreq:533000 
		    Target LITTLE CPU current CPUfreq: 1844000


In [53]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

04:19:53  INFO    : #### Setup FTrace
04:19:58  INFO    : #### Start energy sampling
04:19:59  INFO    : #### Start RTApp execution
04:19:59  INFO    : Workload execution START:
04:19:59  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
04:20:11  INFO    : #### Read energy consumption: /home/steven/lisa/lisa-github/results/20190308_150911/energy.json
04:20:11  ERROR   : /usr/bin/iio-capture terminated for Device1 (iio:device1)
04:20:11  ERROR   : [Unable to create IIO context
]
04:20:11  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
04:20:11  ERROR   : [Unable to create IIO context
]
04:20:11  INFO    : #### Stop FTrace
04:20:11  INFO    : #### Save FTrace: /home/steven/lisa/lisa-github/results/20190308_150911/trace.dat
04:20:15  INFO    : #### Save platform description: /home/steven/lisa/lisa-github/results/20190308_150911/platform.json


In [54]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

04:20:15  INFO    : Content of the output folder /home/steven/lisa/lisa-github/results/20190308_150911


total 46004
drwxrwxr-x 1 steven steven      310  3月  8 16:20 .
drwxrwxr-x 1 steven steven     2466  3月  8 15:09 ..
-rw-rw-r-- 1 steven steven        2  3月  8 16:20 energy.json
-rw-rw-r-- 1 steven steven        2  3月  8 16:20 energy_stats.json
-rw-rw-r-- 1 steven steven      444  3月  8 16:20 output.log
-rw-rw-r-- 1 steven steven     1526  3月  8 16:20 platform.json
-rw-r--r-- 1 steven steven    77660  3月  8 16:20 rt-app-task_p50-1.log
-rw-r--r-- 1 steven steven    77660  3月  8 16:20 rt-app-task_p50_l-2.log
-rw-r--r-- 1 steven steven    12560  3月  8 16:20 rt-app-task_r20-0.log
-rw-r--r-- 1 steven steven     3750  3月  8 16:20 simple_00.json
-rw-r--r-- 1 steven steven 46915584  3月  8 16:20 trace.dat
drwxrwxr-x 1 steven steven     2664  3月  8 16:11 .trace.dat.cache


# Collected Results

# Trace inspection

In [55]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [56]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50','task_p50_l', "task_r20" ])

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7f4c77886a10> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7f4c7dd27790> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.dequeue_task_fair object at 0x7f4c7dd27dd0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7f4c7dd277d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blo

In [57]:
target.cpufreq.set_all_governors('schedutil')

''

In [58]:
target.cpufreq.get_all_governors()

{'0': 'schedutil',
 '1': 'schedutil',
 '2': 'schedutil',
 '3': 'schedutil',
 '4': 'schedutil',
 '5': 'schedutil',
 '6': 'schedutil',
 '7': 'schedutil'}

In [59]:
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_task:util_avg',
      'sched_load_avg_task:load_avg'
    ],
    filters={
        'comm': ['task_p50']
    },
    drawstyle='steps-post',
    per_line=1,
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_cpu:util_avg',
      'sched_load_avg_cpu:load_avg'
    ],
    filters={
        'cpu': [ 4 ]
    },
    drawstyle='steps-post',
    per_line=1,
    xlim=(1, 16),
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()


""
""


""
""


In [60]:
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_task:util_avg',
      'sched_load_avg_task:load_avg'
    ],
    filters={
        'comm': ['task_p50_l']
    },
    drawstyle='steps-post',
    per_line=1,
    xlim=(1, 16),

#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()
trappy.ILinePlot(
    ftrace,                         # TRAPpy RUN object
    signals=[
      'sched_load_avg_cpu:util_avg',
      'sched_load_avg_cpu:load_avg'
    ],
    filters={
        'cpu': [ 0 ]
    },
    drawstyle='steps-post',
    per_line=1,
    xlim=(1, 16),
#     sync_zoom=True,
#     group='UtilVsFrequency',
    marker = '+').view()

""
""


""
""
